In [ ]:

from elasticsearch import Elasticsearch


In [3]:
import tarfile
import xml.etree.ElementTree as ET
import os
import io
def extract_xml_from_taz(taz_path: str):
    """
    Extrait et parse le fichier CRI XML depuis un fichier TAZ
    La structure est: .taz contient .tar qui contient .xml
    
    Args:
        taz_path: Chemin vers le fichier .taz
        
    Returns:
        Tuple (root XML Element, nom du fichier XML) ou (None, "") si erreur
    """
    try:
        print(f"Ouverture de {os.path.basename(taz_path)}...")
        
        # Ouvrir le fichier .taz (qui contient un .tar)
        with tarfile.open(taz_path, "r:*") as taz:
            # Trouver le fichier .tar à l'intérieur
            membre_tar = None
            for m in taz.getmembers():
                if m.name.endswith(".tar"):
                    membre_tar = m
                    break
            
            if not membre_tar:
                print("⚠ Aucun fichier .tar trouvé dans le .taz")
                return None, ""
            
            print(f"✓ Fichier TAR trouvé: {membre_tar.name}")
            
            # Extraire le .tar en mémoire
            tar_bytes = taz.extractfile(membre_tar).read()
            tar_buffer = io.BytesIO(tar_bytes)
            
            # Ouvrir le .tar depuis la mémoire
            with tarfile.open(fileobj=tar_buffer, mode="r:") as tar:
                # Chercher le fichier CRI XML
                for membre in tar.getmembers():
                    if membre.name.startswith('CRI_') and membre.name.endswith('.xml'):
                        print(f"✓ Fichier XML trouvé: {membre.name}")
                        
                        # Extraire et parser le XML directement en mémoire
                        xml_file = tar.extractfile(membre)
                        xml_content = xml_file.read()
                        
                        # Parser le XML
                        root = ET.fromstring(xml_content)
                        
                        return root, membre.name
                
                print("⚠ Aucun fichier CRI XML trouvé dans le TAR")
                return None, ""
                
    except Exception as e:
        print(f"✗ Erreur lors de l'extraction: {e}")
        import traceback
        traceback.print_exc()
        return None, ""



In [4]:

root, xml_filename = extract_xml_from_taz(taz_path = "../data/raw/2021/AN_2021001.taz")
list_orateur = root.findall('.//Orateur')
len(list_orateur)


Ouverture de AN_2021001.taz...
✓ Fichier TAR trouvé: AN_2021001.tar
✓ Fichier XML trouvé: CRI_20210113_001.xml


785

In [10]:
for i in range(10):
    orateur = list_orateur[len(list_orateur) - i - 1].find('Nom').text
    print(orateur)

M. le président.
M. le président.
M. Olivier Véran,
M. Michel Herbillon.
M. Olivier Véran,
M. le président.
Mme Emmanuelle Ménard.
M. le président.
M. Olivier Véran,
M. Thierry Benoit.


In [ ]:
list_orateur[0].findall('.//Orateur')[0].find('Nom').text

'M. le président.'

In [13]:
list_soussection = root.findall('.//SousSection2')
len(list_soussection)
print(list_soussection[0])

<Element 'SousSection2' at 0x7b9f3c883150>


In [15]:
soussection = list_soussection[0].find('TitreStruct').find('Intitule').text
print(soussection)

Aérodrome de Granville-Mont-Saint-Michel


In [6]:
for i in range(10):
    soussection = list_soussection[i].find('Intitule').text
    print(soussection)

AttributeError: 'NoneType' object has no attribute 'text'